In [6]:
import datetime
import logging
import os
import glob
import pandas as pd

from utils.dialog_manipulation import prepare_dialogs_sorted_by_lang
from utils.dialog_manipulation import add_reply_time, add_subdialogs_ids

In [2]:
# def init_args():
#     parser = argparse.ArgumentParser(description="Step #3.Prepare dialogs data.")
    
#     parser.add_argument(
#         "--dialogs_ids",
#         nargs="+",
#         type=int,
#         help="id(s) of dialog(s) to download, -1 for all",
#         required=True,
#     )
#     parser.add_argument(
#         "--config_path",
#         type=str,
#         help="path to config file",
#         default="config/config.json",
#     )
#     parser.add_argument("--debug_mode", type=int, help="Debug mode", default=0)
#     return parser.parse_args()

## Change all this block to your parameters

In [3]:
# args = init_args()


# DIALOGS_IDS = args.dialogs_ids
# DEBUG_MODE = args.debug_mode


# input dialogs_ids (it should be a list) or -1 for all
DIALOGS_IDS = [-1]
DEBUG_MODE = 0
DIALOG_PATH = os.path.join('data', 'all_dialogs2')
LOGS_PATH = 'logs/project_logs.log'

FINAL_DF_NAME = "final_all2.csv"
PATH_TO_PREPARED_DIALOGS = os.path.join("data", "all_prepared_dialogs2")
PATH_TO_SAVE_GENERAL_DF = os.path.join("data", "all_prepared_dialogs2", FINAL_DF_NAME)

# change to date range in what you want to analyse messages of user_id_get_msg - from START_DATE to END_DATE;
# format "%Y-%m-%d %H:%M:%S"
START_DATE = datetime.datetime(2016, 8, 9, 0, 0, 0)
END_DATE = datetime.datetime(2020, 8, 10, 0, 0, 0)


In [4]:
# !pip install -r requirements.txt
# !pip install -U nlpcube

## TODO:

### 1. sometimes prepare_dialogs_sorted_by_lang function checks 999msgs, sometimes it uses 5msgs. this behaviour should be fixed
### 2. pls add index=False in places where you save DF in the file
### 3. if you make this method to performe in the reasonble time(fast) - you can avoid saving temporary files as it's time consuming proccess(save file)

In [5]:
# !pip install -r requirements.txt
# !pip install -U nlpcube

In [ ]:
if DEBUG_MODE:
    logging.basicConfig(filename=LOGS_PATH, level=logging.DEBUG)

if os.path.isdir(DIALOG_PATH):
    if not os.path.isdir(PATH_TO_PREPARED_DIALOGS):
        os.mkdir(PATH_TO_PREPARED_DIALOGS)

    prepare_dialogs_sorted_by_lang(DIALOGS_IDS, DIALOG_PATH, PATH_TO_PREPARED_DIALOGS,
                                   START_DATE, END_DATE, "add_lang_column")

else:
    logging.error('Dialogs dir does not exist !')

In [ ]:
flag_get_all = 0

if DIALOGS_IDS[0] == -1:
    DIALOGS_IDS = glob.glob(f"{PATH_TO_PREPARED_DIALOGS}/*.csv")
    flag_get_all = 1

In [7]:
DIALOGS_IDS

['data/dialogs_prepared/448703929.csv',
 'data/dialogs_prepared/-411310808.csv',
 'data/dialogs_prepared/-377680544.csv',
 'data/dialogs_prepared/862064268.csv',
 'data/dialogs_prepared/-328284553.csv',
 'data/dialogs_prepared/-362548330.csv',
 'data/dialogs_prepared/619023365.csv',
 'data/dialogs_prepared/566042888.csv',
 'data/dialogs_prepared/875082281.csv',
 'data/dialogs_prepared/818427788.csv',
 'data/dialogs_prepared/721785597.csv',
 'data/dialogs_prepared/138036599.csv',
 'data/dialogs_prepared/730752982.csv',
 'data/dialogs_prepared/153198645.csv',
 'data/dialogs_prepared/-169766999.csv',
 'data/dialogs_prepared/-193231004.csv',
 'data/dialogs_prepared/all.csv',
 'data/dialogs_prepared/-197859086.csv']

## TODO: 
### 4. pls move prepare_dialogs_sorted_by_lang function in the for loop
### 5. fix code in the lines:
    data["reply_btw_sender_time"], data["reply_btw_own_time"] = add_reply_time(data)
    data["subdialog_id"] = add_subdialogs_ids(data)

In [8]:
DIALOGS_IDS

In [ ]:
frames = []

general_df = pd.DataFrame()
general_n_subdialogs = 0
general_dialog_id = 0
len_dialogs = len(DIALOGS_IDS)

for n_dialog_id, dialog_file in enumerate(DIALOGS_IDS):
    dialog_id = dialog_file.split('/')[-1]

    if dialog_id == os.path.join(PATH_TO_PREPARED_DIALOGS, FINAL_DF_NAME):
        continue

    if flag_get_all == 1:
        dialog_id = str(dialog_id)[:-4]

    data = pd.read_csv(dialog_file)

    
    data["reply_btw_sender_time"], data["reply_btw_own_time"] = add_reply_time(data)
    data["subdialog_id"] = add_subdialogs_ids(data)

    data["dialog ID"] = n_dialog_id

    general_df = pd.concat([general_df, data])
    
    print(f"{n_dialog_id + 1} dialogs csv from {len_dialogs} succeeded")


general_df.rename(columns={'id': 'message_id'}, inplace=True)

# general_df = general_df.drop("Unnamed: 0", axis=1)
# general_df = general_df.drop("Unnamed: 0.1", axis=1)

cols = ['dialog ID', "message_id", "date", "from_id", 'to_id', 'fwd_from', 'dialog_language',
        'reply_btw_sender_time', 'reply_btw_own_time', 'subdialog_id', 'message', 'preprocessed_message']

rest_cols = [col for col in general_df.columns if col not in cols]

# place long text columns at the end of df
cols = cols[:-2] + rest_cols + cols[-2:]
general_df = general_df[cols]

In [ ]:
general_df.to_csv(PATH_TO_SAVE_GENERAL_DF, index=False)
